In [1]:
import os 
import sys
cwd = os.getcwd()
sys.path.append(cwd)
import test_data_util

test_data_util.TestClass.setup_class()
threshold = 0.5
test = test_data_util.TestClass()
test.test_generate_bin_mask()
test.test_create_filtered_cooler(threshold)

# os.remove(['test_data_util.cool', f'test_data_util_{thres}filtered.cool'])

/home1/yxiao977/.conda/envs/akita/lib/python3.9/site-packages/cooltools/lib/numutils.py:652: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def iterative_correction_symmetric(
/home1/yxiao977/.conda/envs/akita/lib/python3.9/site-packages/cooltools/lib/numutils.py:727: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def iterative_correction_asymmetric(x, max_iter=10

In [7]:
import bioframe as bf
import datetime
import numpy as np
import pandas as pd
now = datetime.datetime.now()
now = str(now)[2:10].replace('-','')
print(now)

num_bad_conseq = 10
badbins_combined = []
hic_target_files = ['/home1/yxiao977/sc1/test_func/test_create.cool']
genomeName = 'dinoflagellate'


for hic_file in hic_target_files:
    print(hic_file)
    # open genome Hi-C file
    genome_hic_cool = cooler.Cooler(hic_file)    
    binSize = genome_hic_cool.info['bin-size']
    bad_inds =  pd.isnull( genome_hic_cool.bins()['weight'][:].values )
    badbins = bf.merge(
        genome_hic_cool.bins()[:].iloc[ bad_inds ][['chrom','start','end']]) 
    badbins.rename(columns={'chrom':'chr','end':'stop'},inplace=True)
    badbin_lens = (badbins['stop'].values-badbins['start'].values)
    print('num bad bins: ', np.sum(bad_inds),', num Mb',np.sum(badbin_lens)/1e6)
    badbins_combined.append( badbins.iloc[ (badbin_lens > (binSize*num_bad_conseq)) ] )

badbins_combined = pd.concat(badbins_combined)
badbins_combined.sort_values(['chr','start','stop'],inplace=True) #just in case 
badbins_combined = bf.merge(badbins_combined, cols=['chr','start','stop'])

badbin_lens = (badbins_combined['stop'].values - badbins_combined['start'].values)
print('num gaps: ',len(badbins_combined),', num Mb',np.sum(badbin_lens)/1e6)
badbins_combined.to_csv( '/home1/yxiao977/sc1/test_func/'+now+'_gaps_'+genomeName+'_binSize'+str(binSize)+
                        '_numconseq'+str(num_bad_conseq)+'.bed',   
                        sep='\t', index=False, header=False, columns=['chr','start','stop'])

231015
/home1/yxiao977/sc1/test_func/test_create.cool
num bad bins:  21068 , num Mb 105.140693
num gaps:  77 , num Mb 18.146076


/home1/yxiao977/.conda/envs/akita/lib/python3.9/site-packages/bioframe/core/specs.py:150: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  pd.api.types.is_categorical_dtype(chrom_dtype),
/home1/yxiao977/.conda/envs/akita/lib/python3.9/site-packages/bioframe/core/specs.py:150: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  pd.api.types.is_categorical_dtype(chrom_dtype),


In [ ]:
! ./akita_data.py -g ~/sc1/test_func/231015_gaps_dinoflagellate_binSize5000_numconseq10.bed --sample 1 -l 250000 --local -o ~/sc1/train_akita/data/4m_unfiltered --as_obsexp -p 8 -t .1 -v .1 -w 5000 --snap 5000 --stride_train 250000 --stride_test 50000 ~/sc1/train_akita/data/GSE152150_Smic1.1N.fa ~/sc1/train_akita/data/unfiltered_cools.txt

In [18]:
t = "--crop 0 -d 2 -k 0 -w 5000 --clip 2.000000 --as_obsexp /home1/yxiao977/sc1/test_func/test.cool /home1/yxiao977/sc1/train_akita/data/4m_unfiltered/sequences.bed /home1/yxiao977/sc1/train_akita/data/4m_unfiltered/seqs_cov/0.h5'"
t = t.split(" ")
import json
print(json.dumps(t))

["--crop", "0", "-d", "2", "-k", "0", "-w", "5000", "--clip", "2.000000", "--as_obsexp", "/home1/yxiao977/sc1/test_func/test.cool", "/home1/yxiao977/sc1/train_akita/data/4m_unfiltered/sequences.bed", "/home1/yxiao977/sc1/train_akita/data/4m_unfiltered/seqs_cov/0.h5'"]


In [ ]:
import data_util
import cooler

output_path = "/home1/yxiao977/sc1/test_func/test_imap_filtered.cool"
clr = cooler.Cooler("/home1/yxiao977/sc1/train_akita/data/5000res_Dplus_hic.cool")

bin_mask = data_util.generate_bin_mask(
    clr, [data_util.cis_total_ratio_filter(thres=0.5)]
)
data_util.create_filtered_cooler(
    output_path, clr, bin_mask, chunksize=10_000_000, nproc=16
)

In [11]:
import cooler 

clr = cooler.Cooler('/home1/yxiao977/sc1/train_akita/data/5000res_Dplus_hic.cool')
clr2 = cooler.Cooler('/home1/yxiao977/sc1/test_func/5000res_Dplus_0.5filtered.cool')
clr3 = cooler.Cooler('/home1/yxiao977/sc1/test_func/test_imap_filtered.cool')
clr4 = cooler.Cooler('/home1/yxiao977/sc1/test_func/test_imap_filtered.mcool::resolutions/5000')

In [12]:
clr4.bins().fetch('chr85_pilon:60000-65000')

,chrom,start,end,weight
145652,chr85_pilon,60000,65000,NaN


In [7]:
# filtered
clr2.bins().fetch('chr85_pilon:60000-65000')

,chrom,start,end,weight,weight_thres0.5
145652,chr85_pilon,60000,65000,NaN,NaN


In [8]:
# zoomified filtered
clr3.bins().fetch('chr85_pilon:60000-65000')

,chrom,start,end
145652,chr85_pilon,60000,65000


In [17]:
# original
clr2.bins().fetch('chr85_pilon:60000-65000')

,chrom,start,end,weight,weight_thres0.5
145652,chr85_pilon,60000,65000,0.009113,NaN
